# Initialize environment

In [1]:
# !pip install --upgrade pip && pip install "numpy>=2.1,<3" "crewai>=0.51.0" "crewai_tools>=0.1.6" "langchain-community>=0.2.10"

# Warning control
import warnings
from crewai import Agent, Task, Crew
import os
from IPython.display import Markdown

warnings.filterwarnings('ignore')

ENV_OPENAI_API_KEY = "OPENAI_API_KEY"

def get_openai_api_key() -> str:
    """Return the OpenAI API key from the environment or raise with guidance."""
    api_key = os.getenv(ENV_OPENAI_API_KEY)
    if not api_key:
        raise RuntimeError(
            f"Missing {ENV_OPENAI_API_KEY}. Set it in your environment, e.g.,\n"
            f"export {ENV_OPENAI_API_KEY}='YOUR_KEY_HERE'"
        )
    return api_key

# Initialize environment and agent
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Define Agents

In [2]:
def create_planner_agent(*, allow_delegation: bool = False, verbose: bool = True) -> Agent:
    """Create and configure the Content Planner agent."""
    return Agent(
        role="Content Planner",
        goal="Plan engaging and factually accurate content on {topic}",
        backstory="You're working on planning a blog article "
                  "about the topic: {topic}."
                  "You collect information that helps the "
                  "audience learn something "
                  "and make informed decisions. "
                  "Your work is the basis for "
                  "the Content Writer to write an article on this topic.",
        allow_delegation=allow_delegation,
        verbose=verbose,
    )

def create_writer_agent(*, allow_delegation: bool = False, verbose: bool = True) -> Agent:
    """Create and configure the Content Writer agent."""
    return Agent(
        role="Content Writer",
        goal="Write insightful and factually accurate "
             "opinion piece about the topic: {topic}",
        backstory="You're working on writing "
                  "a new opinion piece about the topic: {topic}. "
                  "You base your writing on the work of "
                  "the Content Planner, who provides an outline "
                  "and relevant context about the topic. "
                  "You follow the main objectives and "
                  "direction of the outline, "
                  "as provide by the Content Planner. "
                  "You also provide objective and impartial insights "
                  "and back them up with information "
                  "provide by the Content Planner. "
                  "You acknowledge in your opinion piece "
                  "when your statements are opinions "
                  "as opposed to objective statements.",
        allow_delegation=allow_delegation,
        verbose=verbose,
    )

def create_editor_agent(*, allow_delegation: bool = False, verbose: bool = True) -> Agent:
    """Create and configure the Editor agent."""
    return Agent(
        role="Editor",
        goal="Edit a given blog post to align with "
             "the writing style of the organization. ",
        backstory="You are an editor who receives a blog post "
                  "from the Content Writer. "
                  "Your goal is to review the blog post "
                  "to ensure that it follows journalistic best practices,"
                  "provides balanced viewpoints "
                  "when providing opinions or assertions, "
                  "and also avoids major controversial topics "
                  "or opinions when possible.",
        allow_delegation=allow_delegation,
        verbose=verbose
    )

# Create agents instances

In [3]:
planner = create_planner_agent(allow_delegation=False, verbose=True)
writer = create_writer_agent(allow_delegation=False, verbose=True)
editor = create_editor_agent(allow_delegation=False, verbose=True)

# Define tasks

In [4]:
def create_plan_task(*, agent: Agent) -> Task:
    """Create and configure the Plan task."""
    return Task(
        description=(
            "1. Prioritize the latest trends, key players, "
                "and noteworthy news on {topic}.\n"
            "2. Identify the target audience, considering "
                "their interests and pain points.\n"
            "3. Develop a detailed content outline including "
                "an introduction, key points, and a call to action.\n"
            "4. Include SEO keywords and relevant data or sources."
        ),
        expected_output="A comprehensive content plan document "
            "with an outline, audience analysis, "
            "SEO keywords, and resources.",
        agent=agent,
    )

def create_write_task(*, agent: Agent) -> Task:
    """Create and configure the Write task."""
    return Task(
        description=(
            "1. Use the content plan to craft a compelling "
                "blog post on {topic}.\n"
            "2. Incorporate SEO keywords naturally.\n"
            "3. Sections/Subtitles are properly named "
                "in an engaging manner.\n"
            "4. Ensure the post is structured with an "
                "engaging introduction, insightful body, "
                "and a summarizing conclusion.\n"
            "5. Proofread for grammatical errors and "
                "alignment with the brand's voice.\n"
        ),
        expected_output="A well-written blog post "
            "in markdown format, ready for publication, "
            "each section should have 2 or 3 paragraphs.",
        agent=agent,
    )

def create_edit_task(*, agent: Agent) -> Task:
    """Create and configure the Edit task."""
    return Task(
        description=("Proofread the given blog post for "
                     "grammatical errors and "
                     "alignment with the brand's voice."),
        expected_output="A well-written blog post in markdown format, "
                        "ready for publication, "
                        "each section should have 2 or 3 paragraphs.",
        agent=editor
    )

# Create tasks instances

In [5]:
plan = create_plan_task(agent=planner)
write = create_write_task(agent=writer)
edit = create_edit_task(agent=editor)

# Create the crew

In [6]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

# Running the crew

In [10]:
topic = "Agentic AI"
result = crew.kickoff(inputs={"topic": topic})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4df543c7-af51-4f09-9870-e880c3514f3c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Agentic AI.                         │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Title: Unleashing the Power of Agentic AI: Exploring Trends, Players, and Implications                         │
│                                                                                                                 │
│  Outline:                                                                                                       │
│  I. Introduction                                                                                                │
│      A. Definition and importance of Agentic AI                                                                 │
│      B. Thesis statement showcasing the significance of understanding the latest trends and key players         │
│  II. Latest Trends in Agentic AI                                                                                │
│      A. Explanation of current developments in Agentic AI technologies                                          │
│      B. Examples of how Agentic AI is transforming industries                                                   │
│  III. Key Players in the Agentic AI Landscape                                                                   │
│      A. Profiles of leading companies driving innovation in Agentic AI                                          │
│      B. Case studies illustrating successful implementations of Agentic AI                                      │
│  IV. Noteworthy News in Agentic AI                                                                              │
│      A. Recent advancements and breakthroughs in Agentic AI research                                            │
│      B. Implications of emerging trends on future applications of Agentic AI                                    │
│  V. Target Audience Analysis                                                                                    │
│      A. Professionals in the tech industry seeking insights on AI advancements                                  │
│      B. Business leaders looking to capitalize on Agentic AI for strategic decision-making                      │
│  VI. SEO Keywords                                                                                               │
│      A. Agentic AI trends                                                                                       │
│      B. Leading players in Agentic AI                                                                           │
│      C. Agentic AI news                                                                                         │
│  VII. Resources                                                                                                 │
│      A. Research papers on Agentic AI advancements                                                              │
│      B. Industry reports detailing the latest trends in AI technology                                           │
│      C. Interviews with experts in the Agentic AI field                                                         │
│  VIII. Call to Action                                                                                           │
│      A. Encouragement for readers to stay updated on Agentic AI developments                                    │
│      B. Invitation to explore how Agentic AI can benefit their specific industry or business                    │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: af405c4c-ee1b-4fb4-b1ad-3768b844b105                                                                     │
│  Agent: Content Planner                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Task: 1. Use the content plan to craft a compelling blog post on Agentic AI.                                   │
│  2. Incorporate SEO keywords naturally.                                                                         │
│  3. Sections/Subtitles are properly named in an engaging manner.                                                │
│  4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing             │
│  conclusion.                                                                                                    │
│  5. Proofread for grammatical errors and alignment with the brand's voice.                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 91e5ae3c-95e6-4513-a849-6fa5cedc9935                                                                     │
│  Agent: Content Writer                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Unleashing the Power of Agentic AI: Exploring Trends, Players, and Implications                              │
│                                                                                                                 │
│  ## I. Introduction                                                                                             │
│  In the realm of artificial intelligence, Agentic AI stands out for its ability to make decisions and act       │
│  autonomously, mimicking human agency. Understanding the significance of Agentic AI is crucial in today's       │
│  rapidly evolving technological landscape. The latest trends and key players in this field are shaping the      │
│  future of various industries, from healthcare to finance. Exploring these developments is essential for tech   │
│  enthusiasts and business leaders alike to stay ahead in the AI revolution.                                     │
│                                                                                                                 │
│  ## II. Latest Trends in Agentic AI                                                                             │
│  Agentic AI is witnessing notable advancements, with technologies like reinforcement learning enabling          │
│  machines to learn by interacting with their environment. Industries are embracing Agentic AI for tasks that    │
│  require quick decision-making and adaptability. For instance, in healthcare, Agentic AI is streamlining        │
│  patient care through personalized treatment recommendations and diagnostic accuracy. As industries continue    │
│  to integrate Agentic AI solutions, efficiency and innovation are becoming the norm.                            │
│                                                                                                                 │
│  ## III. Key Players in the Agentic AI Landscape                                                                │
│  Leading companies such as Google DeepMind and OpenAI are at the forefront of driving innovation in Agentic     │
│  AI. Their research and production capabilities are pushing the boundaries of what AI can achieve. Case         │
│  studies showcasing successful implementations of Agentic AI, like autonomous vehicles and predictive           │
│  maintenance systems, highlight the tangible benefits of these technologies. Understanding the key players in   │
│  the Agentic AI landscape is pivotal for businesses aiming to leverage AI for competitive advantage.            │
│                                                                                                                 │
│  ## IV. Noteworthy News in Agentic AI                                                                           │
│  Recent breakthroughs in Agentic AI research are paving the way for exciting applications in various domains.   │
│  From advancements in natural language processing to enhanced decision-making algorithms, the possibilities     │
│  are expanding rapidly. The implications of these emerging trends are not limited to technology; they have      │
│  far-reaching effects on how we interact with AI systems and their integration into society. Staying informed   │
│  about the latest news in Agentic AI is essential for anticipating future applications and challenges.          │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 95dfbbd2-0ba1-4db5-bc12-bb76868c7a11                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4df543c7-af51-4f09-9870-e880c3514f3c                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Unleashing the Power of Agentic AI: Exploring Trends, Players, and Implications                │
│                                                                                                                 │
│  ## I. Introduction                                                                                             │
│  In the realm of artificial intelligence, Agentic AI stands out for its ability to make decisions and act       │
│  autonomously, mimicking human agency. Understanding the significance of Agentic AI is crucial in today's       │
│  rapidly evolving technological landscape. The latest trends and key players in this field are shaping the      │
│  future of various industries, from healthcare to finance. Exploring these developments is essential for tech   │
│  enthusiasts and business leaders alike to stay ahead in the AI revolution.                                     │
│                                                                                                                 │
│  ## II. Latest Trends in Agentic AI                                                                             │
│  Agentic AI is witnessing notable advancements, with technologies like reinforcement learning enabling          │
│  machines to learn by interacting with their environment. Industries are embracing Agentic AI for tasks that    │
│  require quick decision-making and adaptability. For instance, in healthcare, Agentic AI is streamlining        │
│  patient care through personalized treatment recommendations and diagnostic accuracy. As industries continue    │
│  to integrate Agentic AI solutions, efficiency and innovation are becoming the norm.                            │
│                                                                                                                 │
│  ## III. Key Players in the Agentic AI Landscape                                                                │
│  Leading companies such as Google DeepMind and OpenAI are at the forefront of driving innovation in Agentic     │
│  AI. Their research and production capabilities are pushing the boundaries of what AI can achieve. Case         │
│  studies showcasing successful implementations of Agentic AI, like autonomous vehicles and predictive           │
│  maintenance systems, highlight the tangible benefits of these technologies. Understanding the key players in   │
│  the Agentic AI landscape is pivotal for businesses aiming to leverage AI for competitive advantage.            │
│                                                                                                                 │
│  ## IV. Noteworthy News in Agentic AI                                                                           │
│  Recent breakthroughs in Agentic AI research are paving the way for exciting applications in various domains.   │
│  From advancements in natural language processing to enhanced decision-making algorithms, the possibilities     │
│  are expanding rapidly. The implications of these emerging trends are not limited to technology; they have      │
│  far-reaching effects on how we interact with AI systems and their integration into society. Staying informed   │
│  about the latest news in Agentic AI is essential for 

In [11]:
Markdown(getattr(result, "raw", str(result)))

# Unleashing the Power of Agentic AI: Exploring Trends, Players, and Implications

## I. Introduction
In the realm of artificial intelligence, Agentic AI stands out for its ability to make decisions and act autonomously, mimicking human agency. Understanding the significance of Agentic AI is crucial in today's rapidly evolving technological landscape. The latest trends and key players in this field are shaping the future of various industries, from healthcare to finance. Exploring these developments is essential for tech enthusiasts and business leaders alike to stay ahead in the AI revolution.

## II. Latest Trends in Agentic AI
Agentic AI is witnessing notable advancements, with technologies like reinforcement learning enabling machines to learn by interacting with their environment. Industries are embracing Agentic AI for tasks that require quick decision-making and adaptability. For instance, in healthcare, Agentic AI is streamlining patient care through personalized treatment recommendations and diagnostic accuracy. As industries continue to integrate Agentic AI solutions, efficiency and innovation are becoming the norm.

## III. Key Players in the Agentic AI Landscape
Leading companies such as Google DeepMind and OpenAI are at the forefront of driving innovation in Agentic AI. Their research and production capabilities are pushing the boundaries of what AI can achieve. Case studies showcasing successful implementations of Agentic AI, like autonomous vehicles and predictive maintenance systems, highlight the tangible benefits of these technologies. Understanding the key players in the Agentic AI landscape is pivotal for businesses aiming to leverage AI for competitive advantage.

## IV. Noteworthy News in Agentic AI
Recent breakthroughs in Agentic AI research are paving the way for exciting applications in various domains. From advancements in natural language processing to enhanced decision-making algorithms, the possibilities are expanding rapidly. The implications of these emerging trends are not limited to technology; they have far-reaching effects on how we interact with AI systems and their integration into society. Staying informed about the latest news in Agentic AI is essential for anticipating future applications and challenges.

## V. Target Audience Analysis
Professionals in the tech industry seeking insights on AI advancements and business leaders looking to capitalize on Agentic AI for strategic decision-making are the primary audience for this blog post. By providing in-depth analysis of trends and players in the Agentic AI space, this content caters to those keen on staying informed about the latest developments in AI technology and its practical implications for business operations.

## Conclusion
Unleashing the power of Agentic AI involves understanding the current trends, key players, and implications of this transformative technology. By staying abreast of the latest news and advancements in Agentic AI, individuals and businesses can harness the potential of AI for innovation and strategic decision-making. As the AI landscape continues to evolve, embracing Agentic AI will be instrumental in driving future growth and success.

## Resources
- For further reading on Agentic AI advancements, refer to research papers in the field.
- Industry reports detailing the latest trends in AI technology offer valuable insights into the Agentic AI landscape.
- Interviews with experts in the Agentic AI field provide firsthand perspectives on the future of autonomous decision-making systems.

## Call to Action
Stay updated on Agentic AI developments and explore how this technology can benefit your specific industry or business. Embrace the power of Agentic AI and unlock new opportunities for growth and innovation.